<a href="https://colab.research.google.com/github/odesay97/MachineLearing_Class/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 트리의 앙상블

# 정형데이터 vs 비정형데이터
# 정형데이터는 csv파일처럼 구조를 가지고 있는데이터
# 비정형데이터는 택스트데이터, 사진, 디지털 음악등 구조를 가지고 있지 않은 데이터

#--> k최근접이웃, 선형회귀, 릿지,라쏘, 다항회귀,로지스틱회귀, 결정트리 등
# 지금까지 배운 방식들은 정형데이터에 적합한 방식임

### 앙상블 학습
 # 정형 데이터를 다루는데 가장 뛰어난 방식

### 랜덤 포레스트
# 앙상블 학습 중 가장 유명하고 안정적인 성능을 제공

# 1. 훈련하기 위한 데이터를 랜덤하게 만듬
#    입력한 훈련 데이터에서 랜덤하게 샘플을 추출함
#    중복된 샘플을 추출할 수 있음
#    이렇게 만들어진 샘플을 부트스트랩 샘플이라함
#    기본적으로 훈련세트의 크기와 동일하게 설정됨
# 샘플생성 랜덤 + 모델생성 랜덤 + 특성선택 랜덤


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 랜덤포레스트 사용 -> 앙상블패키지
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 특성 중요도 -> 특성 중요도가 좀더 평균적이 됨 ( 2번쨰 특성의 중요도가 낮아짐 )
# 왜? -> 랜덤포레스트에서 특성집합에서 루트(특성개수)만큼 랜덤 선택해서 분할선택하는 방식이 있기 때문에
# 각특성이 골고루 영양력을 받음
rf.fit(train_input, train_target)
print(rf.feature_importances_)


# OOB -> 랜덤 과정에서 선택되지 않은 샘플 -> 나중에 테스트샘플로 씀
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

######## 엑스트라 트리
# 랜덤포레스트와 비슷
# 부트스트랩 샘플을 사용하지 않고, 각 결정트리를 만들 때 전체 훈련 세트를 사용
# 노드를 분할할 때, 최적을 찾지 않고 무작위로 감
# --> 엑스트라 트리가 무작위성이 높기 때문에 더 많은 양을 훈련해야함 but 랜덤이라서 더 훈련속도는 빠름

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

#---------------------------------------------------------------------
# 앙상블의 방법
# --> 사진 볼것
# 배깅 -> 병렬로
# 부스팅 -> 순서대로

# 그레디언트 부스팅
# 깊이가 얕은 결정트리를 사용하여 이전트리의 오차를 보완하는 방식
# 트리의 형태에서 경사하강법의 개념을 사용
# -> 사진 보셈
# 얕은 학습 -> 결정트리의 개수를 늘려도 과대적합에 매우 강함
# -> 일반적으로 랜덤포레스트보다 성능이 더 높지만, 훈련시간이 오래걸림


from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.feature_importances_)


# 히스토그램 기반 그레디언트 부스팅
# 히스토그램? -> 데이터를 일정한 구간으로 쪽서 막대그래프로 표현한 것
# 예를 들어 특성이 512개 있다면 -> 특성 1,2를 1로 특성 3,4를 2로 축소
# 256번만 계산하면되므로 효율적
# ==> 256개의 특성으로 줄임


# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)

# 기존의 방식들과 다르게 히스토그램 기반 그레디언트 부스팅은 permutation_importance을 통해 특성 중요도를 계산해서 산출해야한다.
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)


result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

hgb.score(test_input, test_target)

# XGBoost
#사이킷 런의 cross_validate() 함수 사용가능
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))


#LightGBM

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406
0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]
0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]
0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923